In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import keras
import PIL
from PIL import Image, ImageFilter
import numpy as np
import os
import pathlib
import glob


# This code takes jpg's, preps them for the model, and writes them to a tfrecord
## I've spent a bunch of time working with code like this lately so I really don't want to document it rn. Here's a very very brief rundown:

<ul>
    <li>TFRecords are desirable for a few reasons. They're supposed to be fast, and they let you use massive datasets w/o loading them into memory all at once</li>
    <li>The catch is that they can be a pain to use. This is because they only accept limited datatypes (integers, floats, strings, and bytes). You'll notice that arrays are not listed as data that these records can hold.</li>
    <li>So, first you need to turn your numpy arrays into binary strings, then write those strings to the tfrecord. In hindsight it really isn't a huge pain, there's just a bunch of weird TF functions you need to work with and the documentation isn't super general.</li>
    <li>https://tensorflow.google.cn/tutorials/load_data/tfrecord?hl=en</li>
</ul>

In [2]:
# Helper Functions
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
IMAGE_SIZE = [512, 512]

x_dir = '/home/tom/data/frames/both/x'
y_dir = '/home/tom/data/frames/both/y'

x_filenames = glob.glob(x_dir+'/*')
x_filenames.sort()
y_filenames = glob.glob(y_dir+'/*')
y_filenames.sort()


print("-"*20)
print("X Filenames Length: ", len(x_filenames))
print("Y Filenames Length: ", len(y_filenames))
print("-"*20)

--------------------
X Filenames Length:  2001
Y Filenames Length:  2001
--------------------


In [4]:
# Create tfrecord:
outs = '/home/tom/data/road_1frame_predictor/stacked_roadframes.tfrecord'
with tf.io.TFRecordWriter(outs) as writer:
    # I'm just using 140 datapoints here
    # My model is overfit and I don't care
    for index in range(150-10):
        i = index + 5
        x = np.zeros((512,512,3*5))
        for j in range(5):
            if index == 0:
                Image.open(x_filenames[i-j]).show()
            x[:,:,3*j:3*(j+1)] = np.array(Image.open(x_filenames[i-j]).resize((512,512)).convert("RGB"))
        
        y = Image.open(y_filenames[i]).resize((512,512)).convert("RGB")

        x_array = x/255
        y_array = np.array(y)/255
        
        
        x_array = x_array.astype('float64')
        y_array = y_array.astype('float64')

        x_string = (x_array).tostring()
        y_string = (y_array).tostring()

        x_bytes = tf.compat.as_bytes(x_string)
        y_bytes = tf.compat.as_bytes(y_string)

        x_feature = _bytes_feature(x_bytes)
        y_feature = _bytes_feature(y_bytes)

        feature = {
            'image':x_feature,
            'label':y_feature
        }

        feat = tf.train.Features(feature=feature)
        example = tf.train.Example(features=feat)
        writer.write(example.SerializeToString())

hold
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512, 512, 15)
(512,

# The rest of this code just verifies that the images were written correctly

In [6]:
# Read tfrecord

def _parse_function(proto):
    keys_to_features = {
        'image':tf.io.FixedLenFeature([], tf.string),
        'label':tf.io.FixedLenFeature([], tf.string)
    }
    
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    parsed_features['image'] = tf.io.decode_raw(parsed_features['image'], tf.float64)
    parsed_features['label'] = tf.io.decode_raw(parsed_features['label'], tf.float64)
    
    print(parsed_features['image'])
    
    parsed_features['image'] = tf.reshape(parsed_features['image'], [512,512,15])
    parsed_features['label'] = tf.reshape(parsed_features['label'], [512,512,3])
    
    print(parsed_features['image'])
    
    return parsed_features['image'], parsed_features['label']

def load_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    
    dataset = dataset.map(_parse_function, 1)
    
    return dataset

def get_dataset(path):
    dataset = load_dataset(path)
    
    dataset = dataset.shuffle(100)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(10)
    
    return dataset

ds = get_dataset('/home/tom/data/road_1frame_predictor/stacked_roadframes.tfrecord')

Tensor("DecodeRaw:0", shape=(None,), dtype=float64)
Tensor("Reshape:0", shape=(512, 512, 15), dtype=float64)


In [8]:
xbatch, ybatch = next(iter(ds))


In [12]:
x = xbatch[1]
y = ybatch[1]

print(x.shape)
firstframe = x[:,:,:3]
lastframe = x[:,:,12:]
print(firstframe.shape)
print(lastframe.shape)
print(y.shape)

Image.fromarray((firstframe.numpy()*255).astype('uint8')).show()
Image.fromarray((lastframe.numpy()*255).astype('uint8')).show()
Image.fromarray((y.numpy()*255).astype('uint8')).show()

(512, 512, 15)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
